In [1]:
### Evaluate colocalization results
### Load the results saved in R2 script and check out some summaries

# Libraries

In [2]:
source('MS1_Libraries.r')


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ readr   2.1.2
✔ tibble  3.1.7     ✔ purrr   0.3.4
✔ tidyr   1.2.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ data.table::between() masks dplyr::between()
✖ dplyr::filter()       masks stats::filter()
✖ data.table::first()   masks dplyr::first()
✖ dplyr::lag()          masks stats::lag()
✖ data.table::last()    masks dplyr::last()
✖ purrr::transpose()    masks data.table::transpose()


Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


The following objects are mas

# Parameters

In [3]:
path<-""
outdir<-""

In [4]:
cell_type_var = c("CD4T","CD8T","monocyte","NK","B","DC")
# c("CD4T","CD8T","monocyte","NK","B","DC")

# Data 

## Summaries

In [5]:
### EQTL coloclaization summaries

In [6]:
eqtl_summary = read.table(paste0(path, '/colocalization_results/EQTL_summary_update.csv'), row.names = NULL, sep = ",", header = TRUE)

In [7]:
eqtl_summary$X = NULL

In [8]:
nrow(eqtl_summary)

[1] 1260

In [9]:
head(eqtl_summary,2)

,parameter,value,trait,identifier
,<chr>,<dbl>,<chr>,<chr>
1,nsnps,2.46200e+03,White blood cell count,DC1MB_TMEM176A
2,PP.H0.abf,8.18261e-04,White blood cell count,DC1MB_TMEM176A


In [10]:
unique(eqtl_summary$trait)

[1] "White blood cell count"     "Crohn's Disease"           
[3] "Inflammatory Bowel Disease" "Multiple Sclerosis"        
[5] "Rheumatoid Arthritis"       "Asthma"                    
[7] "Type_1_Diabetes"

In [11]:
eqtl_summary = unique(eqtl_summary)

In [12]:
duplicates = eqtl_summary %>% group_by(parameter, trait, identifier) %>% count() %>% filter(n>= 2)

In [13]:
unique(duplicates$trait)

character(0)

In [14]:
# Co-EQTL colocalization summaries

In [15]:
coeqtl_summary = read.table(paste0(path, '/colocalization_results/COEQTL_summary_update.csv'), row.names = NULL, sep = ",", header = TRUE)

In [16]:
coeqtl_summary$X = NULL

In [17]:
nrow(coeqtl_summary)

[1] 44688

In [18]:
head(coeqtl_summary,2)

,parameter,value,trait,identifier
,<chr>,<dbl>,<chr>,<chr>
1,nsnps,2.461000e+03,White blood cell count,monocyte_TMEM176A___CAPG__TMEM176A
2,PP.H0.abf,2.606173e-02,White blood cell count,monocyte_TMEM176A___CAPG__TMEM176A


In [19]:
coeqtl_summary = unique(coeqtl_summary)

In [20]:
duplicates = coeqtl_summary %>% group_by(parameter, trait, identifier) %>% count() %>% filter(n>= 2)

In [21]:
unique(duplicates$trait)

character(0)

In [22]:
#unique(duplicates$parameter)

In [23]:
head(duplicates,2)

parameter,trait,identifier,n
<chr>,<chr>,<chr>,<int>


In [24]:
### Extract egene

In [25]:
coeqtl_summary$egene = str_extract(coeqtl_summary$identifier, '.*___')
coeqtl_summary$egene = str_replace(coeqtl_summary$egene, '___', '')

# Evaluate summaries

## For EQTLs

In [26]:
head(eqtl_summary,2)

,parameter,value,trait,identifier
,<chr>,<dbl>,<chr>,<chr>
1,nsnps,2.46200e+03,White blood cell count,DC1MB_TMEM176A
2,PP.H0.abf,8.18261e-04,White blood cell count,DC1MB_TMEM176A


In [27]:
## Extract amount of SNPS

In [28]:
n_snps = eqtl_summary[eqtl_summary$parameter == 'nsnps',]
n_snps$overlapping_snps = n_snps$value
n_snps$parameter = NULL
n_snps$value = NULL

In [29]:
#eqtl_summary = eqtl_summary[eqtl_summary$parameter != 'nsnps',]

In [30]:
## Extract cell-type, gene etc

In [31]:
eqtl_summary$cell_type = str_replace(eqtl_summary$identifier, '_.*', '')

In [32]:
eqtl_summary$gene = str_replace(eqtl_summary$identifier, '.*_', '')

In [33]:
unique(eqtl_summary$cell_type)

[1] "DC1MB"       "B1MB"        "NK1MB"       "monocyte1MB" "CD8T1MB"    
[6] "CD4T1MB"

In [34]:
## Add number of SNPS

In [35]:
eqtl_summary = merge(eqtl_summary, n_snps, by.x = c('trait', 'identifier'), by.y = c('trait', 'identifier'))

In [36]:
head(eqtl_summary,2)

,trait,identifier,parameter,value,cell_type,gene,overlapping_snps
,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>
1,Asthma,B1MB_HLA-DQA2,nsnps,3.880000e+02,B1MB,HLA-DQA2,388
2,Asthma,B1MB_HLA-DQA2,PP.H0.abf,1.212062e-25,B1MB,HLA-DQA2,388


In [37]:
### Maximum probability per identifier and trait

In [38]:
max_prob = eqtl_summary[eqtl_summary$parameter != 'nsnps',] %>% group_by(trait, identifier, cell_type, gene) %>% summarise(value = max(value))

`summarise()` has grouped output by 'trait', 'identifier', 'cell_type'. You can
override using the `.groups` argument.


In [39]:
eqtl_summary_filtered = merge(eqtl_summary, max_prob)

In [40]:
overview_h_amounts = eqtl_summary_filtered %>% group_by(parameter, trait) %>% summarise(n = n(), mean_value = mean(value), amount_greater_0.9 = sum(value > 0.9), amount_greater_0.75 = sum(value > 0.75),amount_greater_0.5 = sum(value > 0.5))

`summarise()` has grouped output by 'parameter'. You can override using the
`.groups` argument.


In [41]:
## Add snp information

In [42]:
n_snps = n_snps %>% group_by(trait) %>% summarise(average_overlapping_snps = mean(overlapping_snps), min_overlapping_snps = min(overlapping_snps), max_overlapping_snps = max(overlapping_snps))

In [43]:
#n_snps = unique(n_snps[,c('trait', 'overlapping_snps')])

In [44]:
overview_h_amounts = merge(overview_h_amounts, n_snps, by.x = c('trait'), by.y = c('trait'))

In [45]:
#head(n_snps,3)

In [46]:
head(overview_h_amounts[order(overview_h_amounts$trait, decreasing = TRUE),],5)

,trait,parameter,n,mean_value,amount_greater_0.9,amount_greater_0.75,amount_greater_0.5,average_overlapping_snps,min_overlapping_snps,max_overlapping_snps
,<chr>,<chr>,<int>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>
26,White blood cell count,PP.H0.abf,7,0.7627689,0,6,7,1736.267,517,2639
27,White blood cell count,PP.H1.abf,1,0.8050249,0,1,1,1736.267,517,2639
28,White blood cell count,PP.H2.abf,17,0.9584354,16,17,17,1736.267,517,2639
29,White blood cell count,PP.H3.abf,5,0.9526789,4,5,5,1736.267,517,2639
23,Type_1_Diabetes,PP.H3.abf,12,0.9998213,12,12,12,2284.000,1341,3161


In [47]:
### Inspect interesting hypothesis

In [48]:
parameter_var = 'PP.H4.abf'

In [49]:
eqtl_summary_filtered[(eqtl_summary_filtered$parameter == parameter_var) ,]

,trait,identifier,value,cell_type,gene,parameter,overlapping_snps
,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
1,Asthma,B1MB_HLA-DQA2,0.9997353,B1MB,HLA-DQA2,PP.H4.abf,388
3,Asthma,B1MB_RPS26,0.5349101,B1MB,RPS26,PP.H4.abf,381
6,Asthma,CD4T1MB_HLA-DQA2,0.9999956,CD4T1MB,HLA-DQA2,PP.H4.abf,388
8,Asthma,CD4T1MB_RPS26,0.5349363,CD4T1MB,RPS26,PP.H4.abf,381
11,Asthma,CD8T1MB_HLA-DQA2,1.0000000,CD8T1MB,HLA-DQA2,PP.H4.abf,388
13,Asthma,CD8T1MB_RPS26,0.5350001,CD8T1MB,RPS26,PP.H4.abf,381
16,Asthma,DC1MB_HLA-DQA2,0.6589821,DC1MB,HLA-DQA2,PP.H4.abf,388
18,Asthma,DC1MB_RPS26,0.5349096,DC1MB,RPS26,PP.H4.abf,381
21,Asthma,monocyte1MB_HLA-DQA2,0.9746826,monocyte1MB,HLA-DQA2,PP.H4.abf,388


In [50]:
parameter_var = 'PP.H3.abf'

In [51]:
#trait = c('Rheumatoid Arthritis', 'Type_1_Diabetes', 'Crohn\'s Disease')

In [52]:
eqtl_summary_filtered[(eqtl_summary_filtered$parameter == parameter_var) ,]

,trait,identifier,value,cell_type,gene,parameter,overlapping_snps
,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
31,Crohn's Disease,B1MB_HLA-DQA2,0.9999567,B1MB,HLA-DQA2,PP.H3.abf,4698
36,Crohn's Disease,CD4T1MB_HLA-DQA2,0.9998617,CD4T1MB,HLA-DQA2,PP.H3.abf,4698
41,Crohn's Disease,CD8T1MB_HLA-DQA2,0.9995537,CD8T1MB,HLA-DQA2,PP.H3.abf,4698
46,Crohn's Disease,DC1MB_HLA-DQA2,0.9999567,DC1MB,HLA-DQA2,PP.H3.abf,4698
51,Crohn's Disease,monocyte1MB_HLA-DQA2,0.9999567,monocyte1MB,HLA-DQA2,PP.H3.abf,4698
56,Crohn's Disease,NK1MB_HLA-DQA2,0.9998702,NK1MB,HLA-DQA2,PP.H3.abf,4698
57,Crohn's Disease,NK1MB_RNASET2,0.5016812,NK1MB,RNASET2,PP.H3.abf,2707
61,Inflammatory Bowel Disease,B1MB_HLA-DQA2,0.9263285,B1MB,HLA-DQA2,PP.H3.abf,4700
66,Inflammatory Bowel Disease,CD4T1MB_HLA-DQA2,0.8770284,CD4T1MB,HLA-DQA2,PP.H3.abf,4700


In [53]:
### Save eqtl summary for supplementary

In [54]:
eqtl_supp = eqtl_summary

In [55]:
head(eqtl_supp,2)

,trait,identifier,parameter,value,cell_type,gene,overlapping_snps
,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>
1,Asthma,B1MB_HLA-DQA2,nsnps,3.880000e+02,B1MB,HLA-DQA2,388
2,Asthma,B1MB_HLA-DQA2,PP.H0.abf,1.212062e-25,B1MB,HLA-DQA2,388


In [56]:
eqtl_supp = eqtl_supp[(eqtl_supp$parameter %in% c('PP.H3.abf','PP.H4.abf' )) ,]

In [57]:
eqtl_supp$identifier = NULL

In [58]:
eqtl_supp$egene = eqtl_supp$gene
eqtl_supp$gene = NULL

In [59]:
eqtl_supp$coloc_hypothesis = eqtl_supp$parameter
eqtl_supp$parameter = NULL

In [60]:
eqtl_supp$cell_type = str_replace(eqtl_supp$cell_type, '1MB', '')

In [61]:
eqtl_supp$pp_value = eqtl_supp$value
eqtl_supp$value = NULL

In [62]:
eqtl_supp$amount_overlapping_snps = eqtl_supp$overlapping_snps 
eqtl_supp$overlapping_snps = NULL

In [63]:
head(eqtl_supp,3)

,trait,cell_type,egene,coloc_hypothesis,pp_value,amount_overlapping_snps
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
5,Asthma,B,HLA-DQA2,PP.H3.abf,0.0002647404,388
6,Asthma,B,HLA-DQA2,PP.H4.abf,0.9997352596,388
11,Asthma,B,RNASET2,PP.H3.abf,0.0018053181,1133


In [64]:
write.table(eqtl_supp, file = paste0(path, "/colocalization_results/", "Coloc_EQTL_supp_table.csv"), append =FALSE, sep = ",", row.names = FALSE, col.names =TRUE)

## For Co-EQTLs

### General summaries

In [65]:
head(coeqtl_summary,2)

,parameter,value,trait,identifier,egene
,<chr>,<dbl>,<chr>,<chr>,<chr>
1,nsnps,2.461000e+03,White blood cell count,monocyte_TMEM176A___CAPG__TMEM176A,monocyte_TMEM176A
2,PP.H0.abf,2.606173e-02,White blood cell count,monocyte_TMEM176A___CAPG__TMEM176A,monocyte_TMEM176A


In [66]:
## extract amount of SNPs info

In [67]:
n_snps = coeqtl_summary[coeqtl_summary$parameter == 'nsnps',]

In [68]:
n_snps$overlapping_snps = n_snps$value
n_snps$parameter = NULL
n_snps$value = NULL

In [69]:
## Extract relevant columsn

In [70]:
coeqtl_summary = coeqtl_summary[coeqtl_summary$parameter != 'nsnps',]

In [71]:
coeqtl_summary$cell_type = str_replace(coeqtl_summary$identifier, '_.*', '')

In [72]:
#tail(coeqtl_summary,2)

In [73]:
unique(coeqtl_summary$cell_type)

[1] "monocyte" "CD8T"     "CD4T"     "DC"       "B"        "NK"

In [74]:
coeqtl_summary$gene = str_extract(coeqtl_summary$identifier, '_.*')
coeqtl_summary$gene = str_replace(coeqtl_summary$gene, '.*___', '')

In [75]:
length(unique(coeqtl_summary$gene))

[1] 604

In [76]:
### Add number of snps

In [77]:
head(n_snps,2)

,trait,identifier,egene,overlapping_snps
,<chr>,<chr>,<chr>,<dbl>
1,White blood cell count,monocyte_TMEM176A___CAPG__TMEM176A,monocyte_TMEM176A,2461
7,White blood cell count,monocyte_TMEM176A___PTAFR__TMEM176A,monocyte_TMEM176A,2461


In [78]:
coeqtl_summary = merge(coeqtl_summary, n_snps, by.x = c('trait', 'identifier', 'egene'), by.y = c('trait', 'identifier', 'egene'))

In [79]:
### Check out maximum probabilities

In [80]:
max_value = coeqtl_summary %>% group_by(trait, identifier, cell_type, gene, overlapping_snps, egene) %>% summarise(value = max(value))

`summarise()` has grouped output by 'trait', 'identifier', 'cell_type', 'gene',
'overlapping_snps'. You can override using the `.groups` argument.


In [81]:
coeqtl_summary_filtered = merge(coeqtl_summary, max_value)

In [82]:
head(coeqtl_summary_filtered,2)

,trait,identifier,egene,value,cell_type,gene,overlapping_snps,parameter
,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>
1,Asthma,B_RPS26___EEF1A1__RPS26,B_RPS26,0.3734713,B,EEF1A1__RPS26,381,PP.H0.abf
2,Asthma,B_RPS26___RPL10__RPS26,B_RPS26,0.6338403,B,RPL10__RPS26,381,PP.H4.abf


In [83]:
overview_h_amounts = coeqtl_summary_filtered %>% group_by(parameter, trait, egene ) %>% summarise(n = n(), mean_value = mean(value), amount_greater_0.9 = sum(value > 0.9),  amount_greater_0.75 = sum(value > 0.75),amount_greater_0.5 = sum(value > 0.5),  max_overlap_snps = max(overlapping_snps), min_overlap_snps = min(overlapping_snps), mean_overlap_snps = mean(overlapping_snps))

`summarise()` has grouped output by 'parameter', 'trait'. You can override
using the `.groups` argument.


In [84]:
overview_h_amounts = overview_h_amounts[order(overview_h_amounts$trait),]

In [85]:
amount_coegenes_per_e_gene = overview_h_amounts %>% group_by(egene, trait) %>% summarise(total_n = sum(n))

`summarise()` has grouped output by 'egene'. You can override using the
`.groups` argument.


In [86]:
overview_h_amounts = merge(overview_h_amounts, amount_coegenes_per_e_gene)

In [87]:
#### Inspect the results for eGenes

In [88]:
### All RPS26 - co-egene examples

In [89]:
overview_h_amounts[(overview_h_amounts$egene %in%  c('CD4T_RPS26', 'CD8T_RPS26', 'monocyte_RPS26', 'DC_RPS26', 'NK_RPS26','B_RPS26')) & (overview_h_amounts$parameter %in% c('PP.H4.abf')),]

,trait,egene,parameter,n,mean_value,amount_greater_0.9,amount_greater_0.75,amount_greater_0.5,max_overlap_snps,min_overlap_snps,mean_overlap_snps,total_n
,<chr>,<chr>,<chr>,<int>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>
2,Asthma,B_RPS26,PP.H4.abf,20,0.5448488,0,0,16,381,381,381.0000,35
8,Asthma,CD4T_RPS26,PP.H4.abf,353,0.6166249,0,28,323,381,368,380.7734,372
14,Asthma,CD8T_RPS26,PP.H4.abf,241,0.6293361,0,31,212,381,367,380.6307,293
24,Asthma,monocyte_RPS26,PP.H4.abf,127,0.6724475,0,40,114,381,379,380.9685,132
28,Asthma,NK_RPS26,PP.H4.abf,84,0.6149269,0,7,78,379,379,379.0000,96
105,Rheumatoid Arthritis,B_RPS26,PP.H4.abf,33,0.7385906,4,19,31,878,876,877.9394,35
111,Rheumatoid Arthritis,CD4T_RPS26,PP.H4.abf,370,0.8044940,41,325,368,882,835,881.1541,372
118,Rheumatoid Arthritis,CD8T_RPS26,PP.H4.abf,289,0.8303538,64,247,286,882,831,880.4567,293
123,Rheumatoid Arthritis,DC_RPS26,PP.H4.abf,3,0.6804751,0,0,3,855,855,855.0000,3


In [90]:
overview_h_amounts[(overview_h_amounts$egene %in%  c('CD4T_RPS26', 'CD8T_RPS26', 'monocyte_RPS26', 'DC_RPS26', 'NK_RPS26','B_RPS26')) & (overview_h_amounts$parameter %in% c( 'PP.H3.abf')),]

,trait,egene,parameter,n,mean_value,amount_greater_0.9,amount_greater_0.75,amount_greater_0.5,max_overlap_snps,min_overlap_snps,mean_overlap_snps,total_n
,<chr>,<chr>,<chr>,<int>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>
110,Rheumatoid Arthritis,CD4T_RPS26,PP.H3.abf,2,0.6476383,0,0,2,882,882,882.000,372
133,Type_1_Diabetes,B_RPS26,PP.H3.abf,24,0.7354662,7,12,20,1335,1335,1335.000,35
139,Type_1_Diabetes,CD4T_RPS26,PP.H3.abf,336,0.9025018,234,282,329,1341,1274,1339.973,372
144,Type_1_Diabetes,CD8T_RPS26,PP.H3.abf,266,0.9176685,204,223,258,1341,1268,1339.011,293
150,Type_1_Diabetes,DC_RPS26,PP.H3.abf,1,0.4908023,0,0,0,1297,1297,1297.000,3
154,Type_1_Diabetes,monocyte_RPS26,PP.H3.abf,122,0.9210668,92,109,119,1341,1334,1340.934,132
159,Type_1_Diabetes,NK_RPS26,PP.H3.abf,91,0.9621223,81,87,90,1334,1334,1334.000,96


In [91]:
### All HLA-DQA2  - co-egene examples

In [92]:
#unique(overview_h_amounts$egene )

In [93]:
overview_h_amounts[(overview_h_amounts$egene %in% c('CD4T_HLA-DQA2' , 'CD8T_HLA-DQA2', 'monocyte_HLA-DQA2', 'DC_HLA-DQA2'))  & (overview_h_amounts$parameter %in% c('PP.H4.abf', 'PP.H3.abf')),]

,trait,egene,parameter,n,mean_value,amount_greater_0.9,amount_greater_0.75,amount_greater_0.5,max_overlap_snps,min_overlap_snps,mean_overlap_snps,total_n
,<chr>,<chr>,<chr>,<int>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>
3,Asthma,CD4T_HLA-DQA2,PP.H4.abf,13,0.8605121,5,11,13,387,387,387.0000,16
11,Asthma,CD8T_HLA-DQA2,PP.H4.abf,7,0.8719882,5,5,7,386,386,386.0000,7
18,Asthma,DC_HLA-DQA2,PP.H4.abf,5,0.6734167,0,1,5,358,348,351.8000,13
20,Asthma,monocyte_HLA-DQA2,PP.H4.abf,4,0.7193509,1,2,4,388,388,388.0000,17
32,Crohn's Disease,CD4T_HLA-DQA2,PP.H3.abf,16,0.8674851,8,14,16,4525,4525,4525.0000,16
39,Crohn's Disease,CD8T_HLA-DQA2,PP.H3.abf,7,0.8685034,2,6,7,4491,4383,4475.5714,7
45,Crohn's Disease,DC_HLA-DQA2,PP.H3.abf,9,0.6909695,0,3,9,3930,3758,3810.6667,13
47,Crohn's Disease,monocyte_HLA-DQA2,PP.H3.abf,17,0.9558094,14,17,17,4436,4331,4363.8235,17
57,Inflammatory Bowel Disease,CD4T_HLA-DQA2,PP.H3.abf,16,0.7763020,1,9,16,4527,4527,4527.0000,16


In [94]:
### All SMDT1  - co-egene examples

In [95]:
overview_h_amounts[(overview_h_amounts$egene %in%  c('CD4T_SMDT1', 'CD8T_SMDT1')) & (overview_h_amounts$parameter %in% c('PP.H4.abf', 'PP.H3.abf')),]

trait,egene,parameter,n,mean_value,amount_greater_0.9,amount_greater_0.75,amount_greater_0.5,max_overlap_snps,min_overlap_snps,mean_overlap_snps,total_n
<chr>,<chr>,<chr>,<int>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>


In [96]:
### All TMEM176A  - co-egene examples

In [97]:
overview_h_amounts[(overview_h_amounts$egene %in%  c('monocyte_TMEM176A'))  & (overview_h_amounts$parameter %in% c('PP.H4.abf', 'PP.H3.abf')),]

trait,egene,parameter,n,mean_value,amount_greater_0.9,amount_greater_0.75,amount_greater_0.5,max_overlap_snps,min_overlap_snps,mean_overlap_snps,total_n
<chr>,<chr>,<chr>,<int>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>


In [98]:
### All RNASET2 - co-egene examples

In [99]:
overview_h_amounts[(overview_h_amounts$egene %in%  c('CD4T_RNASET2', 'monocyte_RNASET2')) & (overview_h_amounts$parameter %in% c('PP.H4.abf', 'PP.H3.abf')),]

,trait,egene,parameter,n,mean_value,amount_greater_0.9,amount_greater_0.75,amount_greater_0.5,max_overlap_snps,min_overlap_snps,mean_overlap_snps,total_n
,<chr>,<chr>,<chr>,<int>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>
33,Crohn's Disease,CD4T_RNASET2,PP.H3.abf,2,0.5193770,0,0,1,2707,2707,2707,4
34,Crohn's Disease,CD4T_RNASET2,PP.H4.abf,2,0.5193536,0,0,2,2707,2707,2707,4
48,Crohn's Disease,monocyte_RNASET2,PP.H3.abf,1,0.8485668,0,1,1,4390,4390,4390,1
58,Inflammatory Bowel Disease,CD4T_RNASET2,PP.H3.abf,4,0.5968840,0,0,4,2704,2704,2704,4
72,Inflammatory Bowel Disease,monocyte_RNASET2,PP.H3.abf,1,0.8485287,0,1,1,4391,4391,4391,1
109,Rheumatoid Arthritis,CD4T_RNASET2,PP.H3.abf,4,0.9369664,3,4,4,2031,2031,2031,4


In [100]:
#overview_h_amounts[order(overview_h_amounts$trait),]

In [101]:
### Save supplementary table

In [102]:
coeqtl_supp = coeqtl_summary

In [103]:
#coeqtl_supp[is.na(coeqtl_supp$co_egene),]

In [104]:
head(coeqtl_supp,2)

,trait,identifier,egene,parameter,value,cell_type,gene,overlapping_snps
,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>
1,Asthma,B_RPS26___EEF1A1__RPS26,B_RPS26,PP.H0.abf,0.3734713,B,EEF1A1__RPS26,381
2,Asthma,B_RPS26___EEF1A1__RPS26,B_RPS26,PP.H1.abf,0.1466712,B,EEF1A1__RPS26,381


In [105]:
coeqtl_supp = coeqtl_supp[(coeqtl_supp$parameter %in% c('PP.H3.abf','PP.H4.abf' )) ,]

In [106]:
coeqtl_supp$identifier = NULL

In [107]:
coeqtl_supp$egene = str_replace(coeqtl_supp$egene, '.*_', '')

In [108]:
coeqtl_supp$co_egene = coeqtl_supp$gene
coeqtl_supp$gene = NULL
coeqtl_supp$co_egene = str_replace(coeqtl_supp$co_egene, coeqtl_supp$egene, '')
coeqtl_supp$co_egene = str_replace(coeqtl_supp$co_egene, '__', '')

In [109]:
coeqtl_supp$coloc_hypothesis = coeqtl_supp$parameter
coeqtl_supp$parameter = NULL

In [110]:
coeqtl_supp$cell_type = str_replace(coeqtl_supp$cell_type, '1MB', '')

In [111]:
coeqtl_supp$pp_value = coeqtl_supp$value
coeqtl_supp$value = NULL

In [112]:
coeqtl_supp$amount_overlapping_snps = coeqtl_supp$overlapping_snps 
coeqtl_supp$overlapping_snps = NULL

In [113]:
head(coeqtl_supp,3)

,trait,egene,cell_type,co_egene,coloc_hypothesis,pp_value,amount_overlapping_snps
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
4,Asthma,RPS26,B,EEF1A1,PP.H3.abf,0.04642325,381
5,Asthma,RPS26,B,EEF1A1,PP.H4.abf,0.30739863,381
9,Asthma,RPS26,B,RPL10,PP.H3.abf,0.09736711,381


In [114]:
write.table(coeqtl_supp, file = paste0(path, "/colocalization_results/", "Coloc_COEQTL_supp_table.csv"), append =FALSE, sep = ",", row.names = FALSE, col.names =TRUE)

### Get certain co-eqtl examples (RA - RPS26 H4 coegenes)

In [115]:
## 41 co-eGenes for Rheumatoid Arthritis with strong colocalization signal

In [116]:
coeqtl_summary_filtered$gene = str_replace(coeqtl_summary_filtered$gene, '_', '')

In [117]:
nrow(coeqtl_summary_filtered[(coeqtl_summary_filtered$trait == 'Rheumatoid Arthritis' ) & (coeqtl_summary_filtered$value > 0.9 )& (coeqtl_summary_filtered$parameter == 'PP.H4.abf' ),])

[1] 152

In [118]:
head(coeqtl_summary_filtered)

,trait,identifier,egene,value,cell_type,gene,overlapping_snps,parameter
,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>
1,Asthma,B_RPS26___EEF1A1__RPS26,B_RPS26,0.3734713,B,EEF1A1_RPS26,381,PP.H0.abf
2,Asthma,B_RPS26___RPL10__RPS26,B_RPS26,0.6338403,B,RPL10_RPS26,381,PP.H4.abf
3,Asthma,B_RPS26___RPL11__RPS26,B_RPS26,0.3473292,B,RPL11_RPS26,381,PP.H4.abf
4,Asthma,B_RPS26___RPL13__RPS26,B_RPS26,0.5521870,B,RPL13_RPS26,381,PP.H4.abf
5,Asthma,B_RPS26___RPL18__RPS26,B_RPS26,0.5154468,B,RPL18_RPS26,381,PP.H0.abf
6,Asthma,B_RPS26___RPL21__RPS26,B_RPS26,0.5951100,B,RPL21_RPS26,381,PP.H4.abf


In [121]:
coloc_examples_rheomatoid_arthritis = coeqtl_summary_filtered[(coeqtl_summary_filtered$trait == 'Rheumatoid Arthritis' ) & (coeqtl_summary_filtered$value > 0.9 )& (coeqtl_summary_filtered$parameter == 'PP.H4.abf' ),]

In [122]:
#coloc_examples_rheomatoid_arthritis = coeqtl_summary_filtered[(coeqtl_summary_filtered$trait == 'Rheumatoid Arthritis' ) & (coeqtl_summary_filtered$value > 0.9 )& (coeqtl_summary_filtered$parameter == 'PP.H4.abf' ) & (coeqtl_summary_filtered$egene == 'CD4T_RPS26' ),]

In [123]:
analyze = coloc_examples_rheomatoid_arthritis %>% group_by( gene) %>% count()

In [124]:
coeqtl_summary_filtered[coeqtl_summary_filtered$gene == 'HLA-DPB1_RPS26',]

,trait,identifier,egene,value,cell_type,gene,overlapping_snps,parameter
,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>
194,Asthma,CD4T_RPS26___HLA-DPB1__RPS26,CD4T_RPS26,0.5988243,CD4T,HLA-DPB1_RPS26,381,PP.H4.abf
550,Asthma,CD8T_RPS26___HLA-DPB1__RPS26,CD8T_RPS26,0.4321920,CD8T,HLA-DPB1_RPS26,381,PP.H0.abf
860,Asthma,monocyte_RPS26___HLA-DPB1__RPS26,monocyte_RPS26,0.6660012,monocyte,HLA-DPB1_RPS26,381,PP.H4.abf
1258,Crohn's Disease,CD4T_RPS26___HLA-DPB1__RPS26,CD4T_RPS26,0.8220487,CD4T,HLA-DPB1_RPS26,1112,PP.H2.abf
1614,Crohn's Disease,CD8T_RPS26___HLA-DPB1__RPS26,CD8T_RPS26,0.5654940,CD8T,HLA-DPB1_RPS26,1112,PP.H0.abf
1924,Crohn's Disease,monocyte_RPS26___HLA-DPB1__RPS26,monocyte_RPS26,0.9525225,monocyte,HLA-DPB1_RPS26,1112,PP.H2.abf
2322,Inflammatory Bowel Disease,CD4T_RPS26___HLA-DPB1__RPS26,CD4T_RPS26,0.8474188,CD4T,HLA-DPB1_RPS26,1110,PP.H2.abf
2678,Inflammatory Bowel Disease,CD8T_RPS26___HLA-DPB1__RPS26,CD8T_RPS26,0.5830098,CD8T,HLA-DPB1_RPS26,1110,PP.H0.abf
2988,Inflammatory Bowel Disease,monocyte_RPS26___HLA-DPB1__RPS26,monocyte_RPS26,0.9819645,monocyte,HLA-DPB1_RPS26,1110,PP.H2.abf


In [125]:
analyze[order(analyze$n, decreasing = TRUE),]

gene,n
<chr>,<int>
AIF1_RPS26,2
C12orf75_RPS26,2
COX7C_RPS26,2
NACA_RPS26,2
RPL11_RPS26,2
RPL4_RPS26,2
RPS10_RPS26,2
RPS24_RPS26,2
RPS26_TIGIT,2


In [126]:
length(unique(coloc_examples_rheomatoid_arthritis$gene))

[1] 141

In [127]:
nrow(coloc_examples_rheomatoid_arthritis %>% group_by(cell_type, gene) %>% count())

[1] 152

In [128]:
head(coloc_examples_rheomatoid_arthritis[order(coloc_examples_rheomatoid_arthritis$value, decreasing = TRUE),],6)

,trait,identifier,egene,value,cell_type,gene,overlapping_snps,parameter
,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>
5255,Rheumatoid Arthritis,NK_RPS26___RPL23__RPS26,NK_RPS26,0.9777943,NK,RPL23_RPS26,879,PP.H4.abf
4839,Rheumatoid Arthritis,CD8T_RPS26___NACA__RPS26,CD8T_RPS26,0.9713088,CD8T,NACA_RPS26,882,PP.H4.abf
4995,Rheumatoid Arthritis,CD8T_RPS26___RPS26__UBA52,CD8T_RPS26,0.9687585,CD8T,RPS26_UBA52,882,PP.H4.abf
5249,Rheumatoid Arthritis,NK_RPS26___RPL15__RPS26,NK_RPS26,0.9687209,NK,RPL15_RPS26,879,PP.H4.abf
4764,Rheumatoid Arthritis,CD8T_RPS26___DNAJB6__RPS26,CD8T_RPS26,0.9685069,CD8T,DNAJB6_RPS26,882,PP.H4.abf
4428,Rheumatoid Arthritis,CD4T_RPS26___FOXP3__RPS26,CD4T_RPS26,0.9669022,CD4T,FOXP3_RPS26,870,PP.H4.abf


In [129]:
coloc_examples_rheomatoid_arthritis %>% group_by(egene) %>% count()

egene,n
<chr>,<int>
B_RPS26,4
CD4T_RPS26,41
CD8T_RPS26,64
monocyte_RPS26,15
NK_RPS26,28


In [130]:
nrow(coloc_examples_rheomatoid_arthritis)

[1] 152

In [131]:
### Mapp positions for coegenes

In [132]:
coloc_examples_rheomatoid_arthritis$cogene = str_replace(coloc_examples_rheomatoid_arthritis$gene, 'RPS26', '')

In [133]:
coloc_examples_rheomatoid_arthritis$cogene = str_replace(coloc_examples_rheomatoid_arthritis$cogene, '__', '')

In [134]:
coloc_examples_rheomatoid_arthritis$cogene = str_replace(coloc_examples_rheomatoid_arthritis$cogene, '_', '')

In [135]:
unique(coloc_examples_rheomatoid_arthritis$cogene)

[1] "RPL11"       "RPL35A"      "RPL41"       "RPLP2"       "ABHD14B"    
  [6] "ADAM19"      "AIF1"        "AKAP13"      "ATP2B1"      "ATP2B4"     
 [11] "C12orf75"    "CD58"        "CHCHD2"      "COX7C"       "CST7"       
 [16] "CYBA"        "EML4"        "ENTPD1"      "FOXP3"       "FTH1"       
 [21] "GALM"        "GK"          "H3F3A"       "HLA-DPB1"    "HLA-DRA"    
 [26] "IGBP1"       "ISG20"       "LINC00493"   "LSM5"        "MIAT"       
 [31] "MIR4435-1HG" "MT2A"        "PRMT2"       "RGS1"        "RNF19A"     
 [36] "SBDS"        "SERF2"       "TCF7"        "TIGIT"       "TMA7"       
 [41] "TMSB10"      "TNFRSF1B"    "UBB"         "YBX1"        "YWHAB"      
 [46] "ABLIM1"      "ANXA2"       "APOBEC3G"    "BHLHE40"     "C12orf57"   
 [51] "C1orf228"    "CALM1"       "CCL4"        "CD55"        "CD81"       
 [56] "CD8B"        "CFL1"        "CMC1"        "DNAJB6"      "EEF1D"      
 [61] "EEF2"        "EIF1"        "EIF3E"       "EIF3K"       "EIF4B"      
 [66] "FAIM3"       "FLNA"        "FOXP1"       "FTL"         "GLTSCR2"    
 [71] "ID2"         "KLRB1"       "NACA"        "NDFIP1"      "NEAT1"      
 [76] "NPM1"        "PFDN5"       "PPA1"        "PPP2R5C"     "PTPN7"      
 [81] "RHOH"        "RIC3"        "RPL17"       "RPL24"       "RPL39"      
 [86] "RPL4"        "RPS10"       "RPS11"       "RPS24"       "RPS4X"      
 [91] "S100A6"      "SH3YL1"      "SLA"         "SMDT1"       "SRSF2"      
 [96] "SRSF7"       "SYNE1"       "TESPA1"      "TOMM7"       "TUBA4A"     
[101] "TXN"         "UBA52"       "UQCRB"       "ZEB2"        "ATP5A1"     
[106] "CD48"        "EIF3L"       "FAU"         "GPX1"        "IPO7"       
[111] "QARS"        "RNASE6"      "RPL7A"       "RPS9"        "S100A11"    
[116] "TKT"         "TNFAIP2"     "ACTB"        "ARPC2"       "B2M"        
[121] "EEF1B2"      "GNB2L1"      "KLRC1"       "RPL15"       "RPL18"      
[126] "RPL23"       "RPL29"       "RPL30"       "RPL34"       "RPL35"      
[131] "RPL37"       "RPL37A"      "RPL38"       "RPL6"        "RPLP0"      
[136] "RPS16"       "RPS20"       "RPS27A"      "RPS3"        "RPS5"       
[141] "RPSA"

In [136]:
### Retrieve gene positions an map to the data

In [137]:
mart = useEnsembl(biomart = "genes", dataset = "hsapiens_gene_ensembl")

In [138]:
geneSet =  unique(coloc_examples_rheomatoid_arthritis$cogene)

In [139]:
resultTable = biomaRt::getBM(attributes = c("start_position","end_position","description", 'hgnc_symbol', 'chromosome_name'),       
                  filters    = "hgnc_symbol",       
                  values     = geneSet,         
                  mart       = mart)     

In [140]:
nrow(resultTable)

[1] 170

In [141]:
length(unique(resultTable$hgnc_symbol))

[1] 132

In [142]:
### Filter out duplicate mappings

In [143]:
filter = resultTable %>% group_by(hgnc_symbol) %>% count() %>% filter(n >= 2)

In [144]:
filter = filter$hgnc_symbol

In [145]:
resultTable = resultTable[!resultTable$hgnc_symbol %in% filter,]

In [146]:
unique(resultTable$chromosome_name)

[1] "3"  "10" "7"  "5"  "15" "22" "2"  "12" "1"  "14" "11" "20" "16" "19" "17"
[16] "8"  "X"  "21" "4"  "18" "9"  "6"

In [147]:
head(resultTable,2)

,start_position,end_position,description,hgnc_symbol,chromosome_name
,<int>,<int>,<chr>,<chr>,<chr>
1,51968510,51983409,abhydrolase domain containing 14B [Source:HGNC Symbol;Acc:HGNC:28235],ABHD14B,3
2,114431112,114768061,actin binding LIM protein 1 [Source:HGNC Symbol;Acc:HGNC:78],ABLIM1,10


In [148]:
head(coloc_examples_rheomatoid_arthritis,2)

,trait,identifier,egene,value,cell_type,gene,overlapping_snps,parameter,cogene
,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>
4259,Rheumatoid Arthritis,B_RPS26___RPL11__RPS26,B_RPS26,0.9035670,B,RPL11_RPS26,878,PP.H4.abf,RPL11
4269,Rheumatoid Arthritis,B_RPS26___RPL35A__RPS26,B_RPS26,0.9008136,B,RPL35A_RPS26,878,PP.H4.abf,RPL35A


In [149]:
coloc_examples_rheomatoid_arthritis = merge(coloc_examples_rheomatoid_arthritis, resultTable, by.x = 'cogene', by.y = 'hgnc_symbol', all.x = TRUE)

In [150]:
head(coloc_examples_rheomatoid_arthritis[is.na(coloc_examples_rheomatoid_arthritis$chromosome_name),])

,cogene,trait,identifier,egene,value,cell_type,gene,overlapping_snps,parameter,start_position,end_position,description,chromosome_name
,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<int>,<int>,<chr>,<chr>
5,AIF1,Rheumatoid Arthritis,CD4T_RPS26___AIF1__RPS26,CD4T_RPS26,0.9269214,CD4T,AIF1_RPS26,882,PP.H4.abf,NA,NA,NA,NA
6,AIF1,Rheumatoid Arthritis,CD8T_RPS26___AIF1__RPS26,CD8T_RPS26,0.9590870,CD8T,AIF1_RPS26,882,PP.H4.abf,NA,NA,NA,NA
13,ATP5A1,Rheumatoid Arthritis,monocyte_RPS26___ATP5A1__RPS26,monocyte_RPS26,0.9595247,monocyte,ATP5A1_RPS26,882,PP.H4.abf,NA,NA,NA,NA
14,B2M,Rheumatoid Arthritis,NK_RPS26___B2M__RPS26,NK_RPS26,0.9481042,NK,B2M_RPS26,879,PP.H4.abf,NA,NA,NA,NA
19,C1orf228,Rheumatoid Arthritis,CD8T_RPS26___C1orf228__RPS26,CD8T_RPS26,0.9100051,CD8T,C1orf228_RPS26,881,PP.H4.abf,NA,NA,NA,NA
21,CCL4,Rheumatoid Arthritis,CD8T_RPS26___CCL4__RPS26,CD8T_RPS26,0.9169790,CD8T,CCL4_RPS26,882,PP.H4.abf,NA,NA,NA,NA


In [151]:
#head(coloc_examples_rheomatoid_arthritis,4)

In [152]:
nrow(coloc_examples_rheomatoid_arthritis)

[1] 152

In [153]:
write.table(coloc_examples_rheomatoid_arthritis, file = paste0(path, "/colocalization_results/", "CD4T_RPS26_Rheomatoid_Arthritis_Colocalization_CoeGenes.csv"), append =FALSE, sep = ",", row.names = FALSE, col.names =TRUE)